# Numba: fast 'for' loops in python

Author: Pierre Ablin, Mathurin Massias




Numba is a Python package that does Just In Time compilation. It can greatly accelerate Python `for` loops. It implements most Python/Numpy operations.

To install it, simply do `conda install numba` or `pip install numba`. Be sure to have an up-to-date version:`pip install --upgrade numba`.

## First example

Say you want to compute $\sum_{i=1}^n \frac{1}{i^2}$. The following code does it in pure Python:

In [1]:
def sum_python(n):
    output = 0.
    for i in range(1, n + 1):
        output += 1. / i ** 2
    return output

In [2]:
%timeit sum_python(10000)

2.88 ms ± 63.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Numpy
To accelerate this loop, you can vectorize it using Numpy:

In [3]:
import numpy as np


def sum_numpy(n):
    return np.sum(1. / np.arange(1, n + 1) ** 2)

In [4]:
%timeit sum_numpy(10000)

34.1 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Numba

You can also use the `@njit` decorator from Numba. Simply put it on top of the python function:

In [5]:
from numba import njit

@njit
def sum_numba(n):
    output = 0.
    for i in range(1, n + 1):
        output += 1. / i ** 2
    return output

In [6]:
%timeit sum_numba(10000)

12.7 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Orders of magnitude faster than pure Python code, and also (for this example) faster than Numpy !

## Second example: stochastic gradients

Numba can be very handy when coding a stochastic algorithm. Indeed, computing a stochastic gradient can be a very fast operation, hence coding a `for` loop over it in pure Python can slow the code down. 

Take the ridge regression $ \min f(x) = \frac 1n \sum_{i=1}^n f_i(x)$ where:

$$f_i(x) = \frac{1}{2}(a_i^\top x- b_i)^2 + \frac \lambda 2 \|x\|_2^2$$

We have the stochastic gradients: $\nabla f_i(x) = (a_i^\top x - b_i) a_i + \lambda x$,
and the full batch gradient: $\nabla f(x) = \frac1n A^{\top}(A x - b) + \lambda x$

In [7]:
n, p = 100, 100

A = np.random.randn(n, p)
b = np.random.randn(n)

lam = 0.1
x = np.zeros(p)

### Numpy

In [8]:
def grad_i(x, i, A, b, lam):
    ai = A[i]
    return (np.dot(ai, x) - b[i]) * ai + lam * x


def sgd(x, max_iter, step, A, b, lam):
    n, _ = A.shape
    for i in range(max_iter):
        x -= step * grad_i(x, i % n, A, b, lam)
    return x

In [9]:
%timeit grad_i(x, 0, A, b, lam)

4.22 µs ± 225 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
%timeit sgd(x, 1000, 0.0001, A, b, lam)

5.86 ms ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Numba

In [11]:
@njit
def grad_i(x, i, A, b, lam):
    ai = A[i]
    return (np.dot(ai, x) - b[i]) * ai + lam * x


@njit
def sgd(x, max_iter, step, A, b, lam):
    n, _ = A.shape
    for i in range(max_iter):
        x -= step * grad_i(x, i % n, A, b, lam)
    return x

In [12]:
%timeit grad_i(x, 0, A, b, lam)

1.02 µs ± 8.88 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%timeit sgd(x, 1000, 0.0001, A, b, lam)

328 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
